In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [3]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '20-12-2021' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                    AND ev_ebitda > 12 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,EZTC3,"-R$ 1.011.560.000,00","R$ 20,57",9.39,19.34,13.15,49.6,"R$ 497.298.000,00","R$ 145.247.000,00"
1,SOJA3,"-R$ 121.118.000,00","R$ 15,77",13.80,5.35,12.08,15.3,"R$ 133.826.000,00","R$ 88.087.000,00"
2,DOHL3,"-R$ 2.964.000,00","R$ 19,00",10.58,9.28,12.95,19.8,"R$ 135.866.000,00","R$ 28.598.000,00"
3,HAGA3,"-R$ 2.862.580,00","R$ 6,68",7.92,-5.57,12.26,22.3,"R$ 10.037.000,00","R$ 1.246.230,00"


In [10]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""

# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
